In [1]:
import json
import codecs
import sys
import pymongo

# ------------------------------------------
# FUNCTION 1: most_popular_cuisine
# ------------------------------------------
def most_popular_cuisine(my_collection):
    
    # total number of restaurants in New York
    total_restaurants = my_collection.count_documents({})
    
    # creating pipeline to aggregate data from my_collection to 
    # get the cuisine with the highest number of restaurants in New York
    most_popular_cuisine = list(my_collection.aggregate([
        # grouping and counting for each cuisine
        {"$group": {"_id": "$cuisine", "n_restaurants" : {"$sum": 1}}},
        # descending sorting for each of cuisine's number of restaurants
        {"$sort": {"n_restaurants": -1}},
        # getting the first result, i.e. the cuisine with the highest number of restaurants
        {"$limit": 1}
    ]))
    
    # getting the name of cuisine with the highest number of restaurants
    cuisine_name = most_popular_cuisine[0]['_id']
    
    # getting the number of restaurants for the most popular cuisine
    no_restaurants = most_popular_cuisine[0]['n_restaurants'] 
    
    # calculating the most popular cuisine's restanrant ratio with the total number of restaurants in New York
    ratio_cuisine = no_restaurants / total_restaurants * 100

    return cuisine_name, ratio_cuisine


# ------------------------------------------
# FUNCTION 2: ratio_per_borough_and_cuisine
# ------------------------------------------
def ratio_per_borough_and_cuisine(my_collection, cuisine):
    
    # list of boroughs in an ascending order
    boroughs = sorted(my_collection.distinct('borough'))
    
    #... 
    cuisine_restaurants_by_boroughs = list(my_collection.aggregate([
        # filtering/matching the "cuisine" key with the most popular cuisine from "most_popular_cuisine()"
        {"$match": {"cuisine": cuisine}},

        # counting restaurants for each borough
        {"$group": {"_id": "$borough", "n_restaurants" : {"$sum": 1}}},

        # sorting results by boroughs' names in an ascending order
        {"$sort": {"_id": 1}}
    ]))
    
    #...
    all_restaurants_by_boroughs = list(my_collection.aggregate([
        # counting restaurants for each borough
        {"$group": {"_id": "$borough", "n_restaurants" : {"$sum": 1}}},

        # sorting results by boroughs' names in an ascending order
        {"$sort": {"_id": 1}}
    ]))
    
    
    # getting a list of number of the "most-popular-cuisine" restaurants for each borough 
    cuisine_boroughs = [i['n_restaurants'] for i in cuisine_restaurants_by_boroughs]

    # getting a list of number of all-kind-of restaurants for each borough 
    all_boroughs = [i['n_restaurants'] for i in all_restaurants_by_boroughs]
    
    # getting a list of ratio of the "most-popular-cuisine" restaurants to the number of all restaurants in each borough
    ratio = [cu/al for cu,al in zip(cuisine_boroughs, all_boroughs)]
    
    # the borough with the lowest ratio 
    potential_borough = boroughs[ratio.index(min(ratio))]
    
    # the lowest ratio
    lowest_ratio = min(ratio) * 100
    
    return potential_borough, lowest_ratio
    
    
    
# ------------------------------------------
# FUNCTION 3: ratio_per_zipcode
# ------------------------------------------
def ratio_per_zipcode(my_collection, cuisine, borough):

    #...
    top_5_most_rest = list(my_collection.aggregate([
        # filtering/matching the "borough" key with the chosen borough from the ratio_per_borough_and_cuisine() 
        {"$match": {"borough": borough}},

        # counting restaurants for all zipcodes in the chosen borough
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}},

        # sorting the restaurant counts in an descending order, i.e. the greatest coming first and the fewest coming last
        {"$sort": {"count": -1}},

        # choosing the top 5 zipcodes with the most restaurants
        {"$limit": 5},

        # sorting zipcodes in an descending order
        # this is important for calculating the ratio of restaurants
        # of the chosen cuisine in the following steps
        {"$sort": {"_id": -1}} 
    ]))
    
    # list of the top 5 postcodes 
    top_5_zip = [i['_id'] for i in top_5_most_rest]
    
    # list of number of all restaurants for the top 5 postcodes
    no_rest_for_top_5 = [i["count"] for i in top_5_most_rest]
    
    #....
    cuis_rest_in_zip = list(my_collection.aggregate([

        # filtering/matching the "cuisine" key with the most popular cuisine from most_popular_cuisine()
        # and the "borough" key with the chosen borough from the ratio_per_borough_and_cuisine() 
        {"$match": {"cuisine": cuisine, "borough": borough}},

        # counting restaurants for all zipcodes in the chosen borough with the chosen cuisine
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}},

        # sorting zipcodes in an descending order
        # this is important for calculating the ratio of restaurants
        # of the chosen cuisine in the following steps
        {"$sort": {"_id": -1}},

    ]))
    
    # list of the number of the "cuisine" restaurants in the top-5 zipcodes having the most of restaurants
    cuis_rest_in_top_5_zip = [i['count'] for i in cuis_rest_in_zip if i['_id'] in top_5_zip]
    
    # list of ratios of the "cuisine" restaurants to all restaurants for top 5 zipcodes
    ratio_top_5 = [cuis / top5 for cuis , top5 in zip(cuis_rest_in_top_5_zip, no_rest_for_top_5)]
    
    # the target zipcode
    target_zip = top_5_zip[ratio_top_5.index(min(ratio_top_5))]
    
    # the lowest ratio 
    ratio = min(ratio_top_5) * 100        

    return target_zip, ratio

# ------------------------------------------
# FUNCTION 4: best_restaurants
# ------------------------------------------
def best_restaurants(my_collection, cuisine, borough, zipcode):
    
    #...
    rest_review_count = list(my_collection.aggregate([

        # filtering/matching the "cuisine" key with the most popular cuisine from most_popular_cuisine(),
        # the "borough" key with the chosen borough from the ratio_per_borough_and_cuisine()
        # and "zipcode" with the chosen zipcodes from the ratio_per_zipcode()
        {"$match": {"address.zipcode": zipcode, "borough": borough,"cuisine": cuisine}},

        # drilling down the 'grade' field
        {"$unwind": "$grades"},

        # counting no. of reviews for each restaurants
        {"$group": {"_id": "$name", "count": {"$sum": 1}}}
    ]))
    
    # list of restaurants having at least 4 reviews
    rest_list = [i['_id'] for i in rest_review_count if i['count'] >= 4]
    
    #...
    target_rest = list(my_collection.aggregate([

        # getting info from restaurants having at least 4 reviews
        {"$match": {"name": {"$in": rest_list}}},

        # drilling down the 'grade' field
        {"$unwind": "$grades"},

        # calculating avarage review scores for each restaurants
        {"$group": {"_id": "$name", "average_score": {"$avg": "$grades.score"}}},

        # sorting restaurants having the average review scores in a descending order
        {"$sort": {"average_score": -1}},

        # getting the top three restaurants having the best average review scores
        {"$limit": 3}
    ]))
    
    # getting restaurants' names and their average review scores
    best, reviews = [i['_id'] for i in target_rest], [i['average_score'] for i in target_rest]
    
    return best, reviews


# ------------------------------------------
# FUNCTION my_main
# ------------------------------------------/
def my_main(database_name, collection_name):
    # 1. We set up the connection to mongos.exe allowing us to access to the cluster
    mongo_client = pymongo.MongoClient()

    # 2. We access to the desired database
    db = mongo_client.get_database(database_name)

    # 3. We access to the desired collection
    collection = db.get_collection(collection_name)

    # 4. What is the kind of cuisine with more restaurants in the city?
    (cuisine, ratio_cuisine) = most_popular_cuisine(collection)
    print("1. The kind of cuisine with more restaurants in the city is", cuisine, "(with a", ratio_cuisine, "percentage of restaurants of the city)")

    # 5. Which is the borough with smaller ratio of restaurants of this kind of cuisine?
    (borough, ratio_borough) = ratio_per_borough_and_cuisine(collection, cuisine)
    print("\n2. The borough with smaller ratio of restaurants of this kind of cuisine is", borough, "(with a", ratio_borough, "percentage of restaurants of this kind)")

    # 6. Which of the 5 biggest zipcodes of the borough has a smaller ratio of restaurants of the cuisine we are looking for?
    (zipcode, ratio_zipcode) = ratio_per_zipcode(collection, cuisine, borough)
    print("\n3. The zipcode of the borough with smaller ratio of restaurants of this kind of cuisine is zipcode =", zipcode, "(with a", ratio_zipcode, "percentage of restaurants of this kind)")

    # 7. Which are the best 3 restaurants (of the kind of cuisine we are looking for) of our zipcode?
    (best, reviews) = best_restaurants(collection, cuisine, borough, zipcode)
    print("\n4. The best three restaurants (of this kind of couisine) at these zipcode are:", best[0], "(with average reviews score of", reviews[0], "),", best[1], "(with average reviews score of", reviews[1], "),", best[2], "(with average reviews score of", reviews[2], ")")

    # 8. Close the client
    mongo_client.close()

# ---------------------------------------------------------------
#           PYTHON EXECUTION
# This is the main entry point to the execution of our program.
# It provides a call to the 'main function' defined in our
# Python program, making the Python interpreter to trigger
# its execution.
# ---------------------------------------------------------------
if __name__ == '__main__':
    # 1. We get the input arguments
    my_database = "test"
    my_collection = "restaurants"

    # 2. We call to my_main
    my_main(my_database, my_collection)

1. The kind of cuisine with more restaurants in the city is American  (with a 24.381876256950193 percentage of restaurants of the city)

2. The borough with smaller ratio of restaurants of this kind of cuisine is Bronx (with a 17.57912745936698 percentage of restaurants of this kind)

3. The zipcode of the borough with smaller ratio of restaurants of this kind of cuisine is zipcode = 10467 (with a 9.036144578313253 percentage of restaurants of this kind)

4. The best three restaurants (of this kind of couisine) at these zipcode are: Kennedy'S Chicken And Pizza (with average reviews score of 15.8 ), V.I.P.'S Cafe (with average reviews score of 12.666666666666666 ), Burger Barn Restaurant (with average reviews score of 12.6 )


### work directly with the collection

In [2]:
mongo_client = pymongo.MongoClient()
# database
db = mongo_client.get_database("test")
# collection
collection = db.get_collection("restaurants")

In [3]:
# count documents in a collection
collection.count_documents({})

25359

In [4]:
# list collection names
db.list_collection_names()

['restaurants']

In [5]:
# retrieve sample document in the collection
collection.find_one()

{'_id': ObjectId('5e90aa0236fad163810b28bc'),
 'address': {'building': '2780',
  'coord': [-73.98241999999999, 40.579505],
  'street': 'Stillwell Avenue',
  'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American ',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
   'grade': 'A',
   'score': 5},
  {'date': datetime.datetime(2013, 6, 5, 0, 0), 'grade': 'A', 'score': 7},
  {'date': datetime.datetime(2012, 4, 13, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2011, 10, 12, 0, 0), 'grade': 'A', 'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}

### Nhap cau 1

In [6]:
# list of cuisines
collection.distinct('cuisine')[:4]

['Afghan', 'African', 'American ', 'Armenian']

In [7]:
# total number of restaurants
total_restaurants = collection.count_documents({})
total_restaurants

25359

In [8]:
most_popular_cuisine = list(collection.aggregate([
    
    # grouping and counting for each cuisine
    {"$group": {"_id": "$cuisine", "n_restaurants" : {"$sum": 1}}},
    
    # descending sorting for each of cuisine's number of restaurants
    {"$sort": {"n_restaurants": -1}},
    
    # getting the first result, i.e. the cuisine with the highest number of restaurants
    {"$limit": 1}
    
]))

In [9]:
most_popular_cuisine

[{'_id': 'American ', 'n_restaurants': 6183}]

In [10]:
cuisine_name = most_popular_cuisine[0]['_id']
cuisine_name

'American '

In [11]:
no_restaurants = most_popular_cuisine[0]['n_restaurants'] 
no_restaurants

6183

In [12]:
ratio_cuisine = no_restaurants / total_restaurants * 100
ratio_cuisine

24.381876256950193

### Nhap cau 2

In [13]:
# list of boroughs in an ascending order
boroughs = sorted(collection.distinct('borough'))
boroughs

['Bronx', 'Brooklyn', 'Manhattan', 'Missing', 'Queens', 'Staten Island']

In [33]:
cuisine = "American "

cuisine_restaurants_by_boroughs = list(collection.aggregate([
    
    # filtering/matching the "cuisine" key with the most popular cuisine from most_popular_cuisine()
    {"$match": {"cuisine": cuisine}},
    
    # counting restaurants for each borough
    {"$group": {"_id": "$borough", "n_restaurants" : {"$sum": 1}}},
    
    # sorting results by boroughs' names in an ascending order
    {"$sort": {"_id": 1}},
    
]))

cuisine_restaurants_by_boroughs

[{'_id': 'Bronx', 'n_restaurants': 411},
 {'_id': 'Brooklyn', 'n_restaurants': 1273},
 {'_id': 'Manhattan', 'n_restaurants': 3205},
 {'_id': 'Missing', 'n_restaurants': 10},
 {'_id': 'Queens', 'n_restaurants': 1040},
 {'_id': 'Staten Island', 'n_restaurants': 244}]

In [15]:
# getting a list of number of the "most-popular-cuisine" restaurants for each borough 
cuisine_boroughs = [i['n_restaurants'] for i in cuisine_restaurants_by_boroughs]
cuisine_boroughs

[411, 1273, 3205, 10, 1040, 244]

In [16]:
all_restaurants_by_boroughs = list(collection.aggregate([
    
    # counting restaurants for each borough
    {"$group": {"_id": "$borough", "n_restaurants" : {"$sum": 1}}},
    
    # sorting results by boroughs' names in an ascending order
    {"$sort": {"_id": 1}},
    
]))
all_restaurants_by_boroughs

[{'_id': 'Bronx', 'n_restaurants': 2338},
 {'_id': 'Brooklyn', 'n_restaurants': 6086},
 {'_id': 'Manhattan', 'n_restaurants': 10259},
 {'_id': 'Missing', 'n_restaurants': 51},
 {'_id': 'Queens', 'n_restaurants': 5656},
 {'_id': 'Staten Island', 'n_restaurants': 969}]

In [17]:
# getting a list of number of all-kind-of restaurants for each borough 
all_boroughs = [i['n_restaurants'] for i in all_restaurants_by_boroughs]
all_boroughs

[2338, 6086, 10259, 51, 5656, 969]

In [18]:
# getting a list of ratio between the "most-popular-cuisine" restaurants with the number of all restaurants in each borough
ratio = [c / a for c, a in zip(cuisine_boroughs, all_boroughs)]
ratio

[0.1757912745936698,
 0.20916858363457114,
 0.31240861682425186,
 0.19607843137254902,
 0.18387553041018387,
 0.2518059855521156]

In [19]:
boroughs[ratio.index(min(ratio))]

'Bronx'

In [20]:
min(ratio) * 100

17.57912745936698

## Nhap cau 3

In [21]:
borough = 'Bronx'

top_5_most_rest = list(collection.aggregate([
    
    # filtering/matching the "borough" key with the chosen borough from the ratio_per_borough_and_cuisine() 
    {"$match": {"borough": borough}},
    
    # counting restaurants for all zipcodes in the chosen borough
    {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}},
    
    # sorting the restaurant counts in an descending order, i.e. the greatest coming first and the fewest coming last
    {"$sort": {"count": -1}},
    
    # choosing the top 5 zipcodes with the most restaurants
    {"$limit": 5},
    
    # sorting zipcodes in an descending order
    # this is important for calculating the ratio of restaurants
    # of the chosen cuisine in the following steps
    {"$sort": {"_id": -1}} 
]))

top_5_most_rest

[{'_id': '10467', 'count': 166},
 {'_id': '10462', 'count': 150},
 {'_id': '10461', 'count': 152},
 {'_id': '10458', 'count': 193},
 {'_id': '10451', 'count': 161}]

In [22]:
# list of the top 5 postcodes 
top_5_zip = [i['_id'] for i in top_5_most_rest]
top_5_zip

['10467', '10462', '10461', '10458', '10451']

In [23]:
# list of number of all restaurants for the given postcodes
no_rest_for_top_5 = [i["count"] for i in top_5_most_rest]
no_rest_for_top_5

[166, 150, 152, 193, 161]

In [24]:
cuisine = "American "
borough = 'Bronx'

cuis_rest_in_zip = list(collection.aggregate([
    
    # filtering/matching the "cuisine" key with the most popular cuisine from most_popular_cuisine()
    # and the "borough" key with the chosen borough from the ratio_per_borough_and_cuisine() 
    {"$match": {"cuisine": cuisine, "borough": borough}},
    
    # counting restaurants for all zipcodes in the chosen borough with the chosen cuisine
    {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}},
    
    # sorting zipcodes in an descending order
    # this is important for calculating the ratio of restaurants
    # of the chosen cuisine in the following steps
    {"$sort": {"_id": -1}},
    
]))

cuis_rest_in_zip

[{'_id': '10475', 'count': 14},
 {'_id': '10474', 'count': 10},
 {'_id': '10473', 'count': 3},
 {'_id': '10472', 'count': 9},
 {'_id': '10471', 'count': 17},
 {'_id': '10470', 'count': 8},
 {'_id': '10469', 'count': 10},
 {'_id': '10468', 'count': 12},
 {'_id': '10467', 'count': 15},
 {'_id': '10466', 'count': 11},
 {'_id': '10465', 'count': 28},
 {'_id': '10464', 'count': 14},
 {'_id': '10463', 'count': 28},
 {'_id': '10462', 'count': 29},
 {'_id': '10461', 'count': 25},
 {'_id': '10460', 'count': 12},
 {'_id': '10459', 'count': 7},
 {'_id': '10458', 'count': 34},
 {'_id': '10457', 'count': 14},
 {'_id': '10456', 'count': 6},
 {'_id': '10455', 'count': 9},
 {'_id': '10454', 'count': 10},
 {'_id': '10453', 'count': 7},
 {'_id': '10452', 'count': 11},
 {'_id': '10451', 'count': 68}]

In [25]:
# list of the number of the "cuisine" restaurants in the top-5 zipcodes having the most of restaurants
cuis_rest_in_top_5_zip = [i['count'] for i in cuis_rest_in_zip if i['_id'] in top_5_zip]
cuis_rest_in_top_5_zip

[15, 29, 25, 34, 68]

In [26]:
# list of ratios of the "cuisine" restaurants to all restaurants for top 5 zipcodes
ratio_top_5 = [cuis / top5 for cuis , top5 in zip(cuis_rest_in_top_5_zip, no_rest_for_top_5)]
ratio_top_5

[0.09036144578313253,
 0.19333333333333333,
 0.16447368421052633,
 0.17616580310880828,
 0.422360248447205]

In [27]:
# the target zipcode
target_zip = top_5_zip[ratio_top_5.index(min(ratio_top_5))]
target_zip

'10467'

In [28]:
# the lowest ratio 
ratio = min(ratio_top_5) * 100
ratio

9.036144578313253

## Nhap cau 4

In [34]:
cuisine = 'American '
borough = 'Bronx'
zipcode = '10467'

#...
rest_review_count = list(collection.aggregate([
    
    # filtering/matching the "cuisine" key with the most popular cuisine from most_popular_cuisine(),
    # the "borough" key with the chosen borough from the ratio_per_borough_and_cuisine()
    # and "zipcode" with the chosen zipcodes from the ratio_per_zipcode()
    {"$match": {"address.zipcode": zipcode, "borough": borough,"cuisine": cuisine}},
    
    # drilling down the 'grade' field
    {"$unwind": "$grades"},
    
    # counting no. of reviews for each restaurants
    {"$group": {"_id": "$name", "count": {"$sum": 1}}}
]))

rest_review_count

[{'_id': 'Woodlawn Cafe', 'count': 6},
 {'_id': 'Burger Barn Restaurant', 'count': 5},
 {'_id': 'Gasolina Bar Lounge', 'count': 2},
 {'_id': 'Zymi Bar & Grill', 'count': 3},
 {'_id': 'Allerton Diner', 'count': 2},
 {'_id': 'Moshulo Golf Course', 'count': 1},
 {'_id': "Kennedy'S Chicken And Pizza", 'count': 5},
 {'_id': 'Coffee Shop', 'count': 5},
 {'_id': 'Kennedy Fried Chicken', 'count': 4},
 {'_id': 'Salud Y Estilo De Vida', 'count': 4},
 {'_id': "Nicky'S Coffee Shop", 'count': 5},
 {'_id': '502 Bar Lounge', 'count': 4},
 {'_id': "Kennedy'S Chicken & Pizza", 'count': 1},
 {'_id': "John'S Luncheonette", 'count': 5},
 {'_id': "V.I.P.'S Cafe", 'count': 6}]

In [35]:
# list of restaurants having at least 4 reviews
rest_list = [i['_id'] for i in rest_review_count if i['count'] >= 4]
rest_list

['Woodlawn Cafe',
 'Burger Barn Restaurant',
 "Kennedy'S Chicken And Pizza",
 'Coffee Shop',
 'Kennedy Fried Chicken',
 'Salud Y Estilo De Vida',
 "Nicky'S Coffee Shop",
 '502 Bar Lounge',
 "John'S Luncheonette",
 "V.I.P.'S Cafe"]

In [38]:
#...
target_rest = list(collection.aggregate([
    
    # getting info from restaurants having at least 4 reviews
    {"$match": {"name": {"$in": rest_list}}},
    
    # drilling down the 'grade' field
    {"$unwind": "$grades"},
    
    # calculating avarage review scores for each restaurants
    {"$group": {"_id": "$name", "average_score": {"$avg": "$grades.score"}}},
    
    # sorting restaurants having the average review scores in a descending order
    {"$sort": {"average_score": -1}},
    
    # getting the top three restaurants having the best average review scores
    {"$limit": 3}
]))

target_rest

[{'_id': "Kennedy'S Chicken And Pizza", 'average_score': 15.8},
 {'_id': "V.I.P.'S Cafe", 'average_score': 12.666666666666666},
 {'_id': 'Burger Barn Restaurant', 'average_score': 12.6}]

In [37]:
# getting restaurants' names and their average review scores
names, reviews = [i['_id'] for i in target_rest], [i['average_score'] for i in target_rest]